In [7]:
# Spark supports Dataframes
# SQL can query this
# Good for DB schemas (exposes JDBC/ODBC server with Hive)
# Trend is to use Dataframes (easier + more interoperable)
# In Scala, you'd use Datasets as untyped
# Can combine User Defined Functions within SQL command

In [8]:
# Import local files from course
from google.colab import files
uploaded = files.upload()

Saving fakefriends.csv to fakefriends.csv


In [9]:
# Installing Spark 3.0.1 with dependencies
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
print('JDK 8 installed')
!wget -q https://www-us.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
print('Spark 3.0.1 installer downloaded')
!tar xf spark-3.0.1-bin-hadoop2.7.tgz
print('Spark 3.0.1 downloaded')
!pip install -q findspark
print('findspark installed')
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"
print('Environment Variables Set')
import findspark
findspark.init()
print('Install Complete')

JDK 8 installed
Spark 3.0.1 installer downloaded
Spark 3.0.1 downloaded
findspark installed
Environment Variables Set
Install Complete


In [10]:
# Using SqparkSQL on Social Media data
from pyspark.sql import SparkSession, Row

# Create a SparkSession for SQL
spark = SparkSession.builder.appName('SparkSQL').getOrCreate()

# Split csv by comma and return schema as structured row
def mapper(line):
  fields = line.split(',')
  return Row(ID = int(fields[0]), name = str(fields[1].encode('utf-8')),
             age = int(fields[2]), numFriends = int(fields[3]))

# Read dataset as RDD
lines = spark.sparkContext.textFile('fakefriends.csv')
people = lines.map(mapper)

# Infer schema and register Dataframe as a table.
# Cache to keep in memory and Create temp view to be used as DB table
schema = spark.createDataFrame(people).cache()
schema.createOrReplaceTempView('people')

# SQL query. Note: people in query same as temp view
print('SQL way')
teens = spark.sql('select * from people where age >= 13 and age <= 19 order by age')

# SQL query result is RDD so use as normal
for teen in teens.collect():
  print(teen)

# Can also use standard Pandas style functions
print('')
print('Pandas style')
schema.groupBy('age').count().orderBy('age').show()

# Stop the SparkContext
spark.stop()

SQL way
Row(ID=106, name="b'Beverly'", age=18, numFriends=499)
Row(ID=115, name="b'Dukat'", age=18, numFriends=397)
Row(ID=341, name="b'Data'", age=18, numFriends=326)
Row(ID=377, name="b'Beverly'", age=18, numFriends=418)
Row(ID=404, name="b'Kasidy'", age=18, numFriends=24)
Row(ID=439, name="b'Data'", age=18, numFriends=417)
Row(ID=444, name="b'Keiko'", age=18, numFriends=472)
Row(ID=494, name="b'Kasidy'", age=18, numFriends=194)
Row(ID=21, name="b'Miles'", age=19, numFriends=268)
Row(ID=52, name="b'Beverly'", age=19, numFriends=269)
Row(ID=54, name="b'Brunt'", age=19, numFriends=5)
Row(ID=133, name="b'Quark'", age=19, numFriends=265)
Row(ID=136, name="b'Will'", age=19, numFriends=335)
Row(ID=225, name="b'Elim'", age=19, numFriends=106)
Row(ID=304, name="b'Will'", age=19, numFriends=404)
Row(ID=366, name="b'Keiko'", age=19, numFriends=119)
Row(ID=373, name="b'Quark'", age=19, numFriends=272)
Row(ID=409, name="b'Nog'", age=19, numFriends=267)
Row(ID=492, name="b'Dukat'", age=19, numFri

In [11]:
from google.colab import files
uploaded = files.upload()

Saving fakefriends-header.csv to fakefriends-header (1).csv


In [12]:
from pyspark.sql import SparkSession, Row

# Create a SparkSession for SQL
spark = SparkSession.builder.appName('SparkSQL').getOrCreate()

# Handle dataset header + take file schema
people = spark.read.option('header','true').option('inferschema','true').csv('fakefriends-header.csv')

# Inferred Schema + Names
people.printSchema()
print('')
people.select('name').show(3)

root
 |-- userID: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- friends: integer (nullable = true)


+--------+
|    name|
+--------+
|    Will|
|Jean-Luc|
|    Hugh|
+--------+
only showing top 3 rows



In [13]:
# Age under 21
age = people.filter(people.age < 21)
people.groupBy('age').count().show(3)
print('')
# Add 10 years
people.select(people.name, people.age + 10).show(3)
# Stop the SparkContext
spark.stop()

+---+-----+
|age|count|
+---+-----+
| 31|    8|
| 65|    5|
| 53|    7|
+---+-----+
only showing top 3 rows


+--------+----------+
|    name|(age + 10)|
+--------+----------+
|    Will|        43|
|Jean-Luc|        36|
|    Hugh|        65|
+--------+----------+
only showing top 3 rows

